# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook: Ensemble of Models


### **Mount google drive**

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install flair
!pip install transformer

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


### **Load Data and Preprocess**

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/sentimentpolarity.csv")
print(df.groupby(['label']).size())
df.head()

label
0    1000
1    1000
dtype: int64


,text,label
0,[ferrera] has the charisma of a young woman wh...,1
1,"both flawed and delayed , martin scorcese's ga...",1
2,"for his first attempt at film noir , spielberg...",1
3,easily one of the best and most exciting movie...,1
4,this director's cut -- which adds 51 minutes -...,0


**Preprocessor to Remove all special characters except emoticons**

In [ ]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['text'][19])
print(preprocessor(df['text'][19]))

the only fun part of the movie is playing the obvious game . you try to guess the order in which the kids in the house will be gored . 
the only fun part of the movie is playing the obvious game you try to guess the order in which the kids in the house will be gored 


In [ ]:
df['text'] = df['text'].apply(preprocessor)

### **Seperate Into Train, Test and Validation Sets**

In [ ]:
df_train=df.iloc[0:int(len(df)*0.85)].reset_index(drop=True)
df_test=df.iloc[int(len(df)*0.85):].reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

df_test, df_eval, sentiment_test, sentiment_eval = train_test_split(df_test['text'], df_test['label'], 
                                                                      random_state=1, test_size=.10, 
                                                                      shuffle=True)


print('Length of train set:',len(df_test),'Length of test set:',len(df_eval))

Length of train set: 270 Length of test set: 30


In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def tokenizer(text):
  return [stemmer.stem(word) for word in text.split()]

### **Load All Models and Predict to prepare for Emsemble Model**

****

**Logistic and Naivbayees**

In [ ]:
from joblib import load
from sklearn.feature_extraction.text import CountVectorizer

tfidf=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/tfidf_logistic.joblib')
model_reg=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/clf_logistic.joblib')

vec_vocabulary=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/vec_naivebayees.joblib')
vec= CountVectorizer(vocabulary=vec_vocabulary)
model_nb=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/clf_naivebayees.joblib')

**LSTM Model**

In [ ]:
import io
import json
from tensorflow import keras

with open('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = keras.preprocessing.text.tokenizer_from_json(data)

model_lstm=keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/model_lstm.h5')

**Flair Model**



In [ ]:
from flair.models import TextClassifier

model_flair=TextClassifier.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/resources/taggers/trec/best-model.pt')

2021-05-03 16:02:07,444 loading file /content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/resources/taggers/trec/best-model.pt


**Bert Model**

In [ ]:
import torch
from transformers import BertForSequenceClassification 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(df.label.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Sentence Polarity Dataset/Models/BERT_ft_epoch8.model'))

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### **Getting Predictions on Test Data Set**

**NaiveBayees and Logistic Models**

In [ ]:
probas=[]
probas_eval=[]

probas.append(model_reg.predict_proba(tfidf.transform(df_test)))
probas.append(model_nb.predict_proba(vec.transform(df_test)))

probas_eval.append(model_reg.predict_proba(tfidf.transform(df_eval)))
probas_eval.append(model_nb.predict_proba(vec.transform(df_eval)))

In [ ]:
predictions=[]
predictions.append(model_reg.predict(tfidf.transform(df_test)))
predictions.append(model_nb.predict(vec.transform(df_test)))


predictions_eval=[]
predictions_eval.append(model_reg.predict(tfidf.transform(df_eval)))
predictions_eval.append(model_nb.predict(vec.transform(df_eval)))

**LSTM Model**

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_length= 500

test_sequences = tokenizer.texts_to_sequences(df_test)
test_sequences = pad_sequences(test_sequences,maxlen =max_seq_length)

lstm_pred=model_lstm.predict(test_sequences)
probas.append(lstm_pred)

In [ ]:
max_seq_length= 500

eval_sequences = tokenizer.texts_to_sequences(df_eval)
eval_sequences = pad_sequences(eval_sequences,maxlen =max_seq_length)

lstm_pred_eval=model_lstm.predict(eval_sequences)
probas_eval.append(lstm_pred_eval)

In [ ]:
preds = np.argmax(lstm_pred, axis=1).flatten()
predictions.append(preds)

preds= np.argmax(lstm_pred_eval, axis=1).flatten()
predictions_eval.append(preds)

**Flair Model**

In [ ]:
from flair.data import Sentence

results=[]
for i in df_test.index:
    sentence=Sentence(df_test[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
    else:
      score=sentence.get_labels()[0].score
    results.append([score,1-score])
probas.append(np.array(results))

In [ ]:
preds = np.argmax(np.array(results), axis=1).flatten()
predictions.append(preds)

In [ ]:
from flair.data import Sentence

results=[]
for i in df_eval.index:
    sentence=Sentence(df_eval[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
    else:
      score=sentence.get_labels()[0].score
    results.append([score,1-score])
probas_eval.append(np.array(results))

In [ ]:
preds = np.argmax(np.array(results), axis=1).flatten()
predictions_eval.append(preds)

**Bert Model**

In [ ]:
from transformers import BertTokenizer

tokenizerbert = BertTokenizer.from_pretrained(
                  'bert-base-uncased',
                  do_lower_case=True) 


encoded_data_test=tokenizerbert.batch_encode_plus(
                        df_test.values,              # Same we are doing for validation set.
                        add_special_tokens=True,
                        return_attention_mask=True,
                        padding='longest',
                        max_length=256,
                        truncation=True,
                        return_tensors='pt')

encoded_data_eval=tokenizerbert.batch_encode_plus(
                        df_eval.values,              # Same we are doing for validation set.
                        add_special_tokens=True,
                        return_attention_mask=True,
                        padding='longest',
                        max_length=256,
                        truncation=True,
                        return_tensors='pt')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

input_ids_test= encoded_data_test['input_ids']
attention_masks_test= encoded_data_test['attention_mask']

dataset_test= TensorDataset(input_ids_test, attention_masks_test,)

dataloader_test = DataLoader(
    dataset_test, 
    sampler=SequentialSampler(dataset_test), 
    batch_size=4
    )

In [ ]:
input_ids_eval= encoded_data_eval['input_ids']
attention_masks_eval= encoded_data_eval['attention_mask']

dataset_eval= TensorDataset(input_ids_eval, attention_masks_eval,)

dataloader_eval = DataLoader(
    dataset_eval, 
    sampler=SequentialSampler(dataset_eval), 
    batch_size=4
    )

In [ ]:
import torch.nn.functional as F

def predict(dataloader_test):
  
    model.eval()
    all_logits = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # since we have no loss, the only thing returned is logits
        logits = outputs[0]
        all_logits.append(logits)
    
    all_logits = torch.cat(all_logits, dim=0)
    preds_flat = np.argmax(all_logits.cpu().numpy(), axis=1).flatten()

    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    # get highest prob dimension as prediction
    
    return preds_flat, probs



In [ ]:
preds, probs=predict(dataloader_test)
probas.append(probs) 

In [ ]:
predictions.append(preds)

In [ ]:
preds, probs=predict(dataloader_eval)
probas_eval.append(probs) 

In [ ]:
predictions_eval.append(preds)

**Concatenate all Predictions on test data to get one row for each record**

In [ ]:
predictions=np.array(predictions)
predictions_eval=np.array(predictions_eval)

In [ ]:
probas=np.array(probas)
probas_eval=np.array(probas_eval)

### **Method 1: Using Probabilities and Weighted Majority Voting**

### **Finding Weights of each Model**

Reference: https://machinelearningmastery.com/weighted-average-ensemble-for-deep-learning-neural-networks/

In [ ]:
import random
from numpy.linalg import norm

weights = [random.uniform(0, 1)for _ in range(5)]
l1norm = norm(weights,1)
weights= weights / l1norm
print(weights)

[0.25835939 0.0856632  0.11778175 0.26556964 0.27262602]


**Calculate Accuracy with Initial Weights**

In [ ]:
from sklearn.metrics import accuracy_score

weightedavg = np.average(probas, axis=0, weights=weights)
result = np.argmax(weightedavg, axis=1)
accuracy_score(result,sentiment_test)

0.8851851851851852

**Find Optimal Weights**

In [ ]:
from numpy.linalg import norm

def loss_func(weights):
  l1norm = norm(weights,1)
  weights= weights / l1norm
  weightedavg = np.average(probas, axis=0, weights=weights)
  result = np.argmax(weightedavg, axis=1)
  return 1 - accuracy_score(result,sentiment_test)

In [ ]:
from scipy.optimize import differential_evolution

bound_w = [(0.0, 1.0)  for _ in range(5)]
result = differential_evolution(loss_func, bound_w, maxiter=1000000, tol=1e-7, disp=True)

differential_evolution step 1: f(x)= 0.111111
differential_evolution step 2: f(x)= 0.111111
differential_evolution step 3: f(x)= 0.111111
differential_evolution step 4: f(x)= 0.111111
differential_evolution step 5: f(x)= 0.111111
differential_evolution step 6: f(x)= 0.107407
differential_evolution step 7: f(x)= 0.107407
differential_evolution step 8: f(x)= 0.107407
differential_evolution step 9: f(x)= 0.107407
differential_evolution step 10: f(x)= 0.107407
differential_evolution step 11: f(x)= 0.107407
differential_evolution step 12: f(x)= 0.107407
differential_evolution step 13: f(x)= 0.107407
differential_evolution step 14: f(x)= 0.107407
differential_evolution step 15: f(x)= 0.107407
differential_evolution step 16: f(x)= 0.107407
differential_evolution step 17: f(x)= 0.107407
differential_evolution step 18: f(x)= 0.107407
differential_evolution step 19: f(x)= 0.107407
differential_evolution step 20: f(x)= 0.107407
differential_evolution step 21: f(x)= 0.107407
differential_evolution

In [ ]:
weights=result['x']
l1norm = norm(weights,1)
final_weights= weights / l1norm
print(final_weights)

[0.08697584 0.27607297 0.03262954 0.30493184 0.29938981]


In [ ]:
weightedavg = np.average(probas, axis=0, weights=weights)
result = np.argmax(weightedavg, axis=1)
print(accuracy_score(result,sentiment_test))

0.8925925925925926


### **Test on Evaluation Set**

In [ ]:
weightedavg_eval = np.average(probas_eval, axis=0, weights=weights)
result_eval = np.argmax(weightedavg_eval, axis=1)
print(accuracy_score(result_eval,sentiment_eval))

0.8333333333333334


### **Method 2: Majority Voting using direct predictions**

In [ ]:
predictions_t=predictions.T
predictions_eval_t=predictions_eval.T

In [ ]:
final_preds=np.array([np.argmax(np.bincount(predictions_t[i],weights=[1,1,1,1,1])) for i in range(predictions_t.shape[0])])
accuracy_score(final_preds,sentiment_test)

0.8333333333333334

In [ ]:
final_preds_eval=np.array([np.argmax(np.bincount(predictions_eval_t[i],weights=[1,1,1,1,1])) for i in range(predictions_eval_t.shape[0])])
accuracy_score(final_preds_eval,sentiment_eval)

0.8333333333333334